In [1]:
import os
# 下面注释掉的适合在非Pycharm 的Juypter Notebook上使用
# import sys
# path = os.path.join(os.path.dirname(os.getcwd()))
# sys.path.append(path) # 将所需要的根目录添加到路径
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from utils.common_utils import printlog, make_dir
from utils.processing_tools.processing import *
from utils.feature_extraction_tools.feature_extraction import emg_feature_extraction

In [2]:
### 参数设置
## 1. 原始文件路径
raw_file_path = 'rawData'
## 2. 受试者和运动类型设置
subjects_list_global = list(['01','02','03','04','05', '31','32','33','34','35'])
# motions_list_global = ['WAK', 'STDUP', 'SITDN', 'UPS', 'DNS', 'KLFT', 'TPTO', 'LLF', 'LLB', 'LLS', 'KLCL'] ## label_encoded = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], onehot_encoded
motions_list_global = ['WAK', 'STDUP', 'SITDN', 'UPS', 'DNS'] ## label_encoded = [0, 1, 2, 3, 4]
## 3. 所使用的原始数据中emg的通道（列）名
emg_name = ['sEMG: tensor fascia lata', 'sEMG: rectus femoris', 'sEMG: vastus medialis',
            'sEMG: semimembranosus', 'sEMG: upper tibialis anterior', 'sEMG: lower tibialis anterior',
            'sEMG: lateral gastrocnemius', 'sEMG: medial gastrocnemius', 'sEMG: soleus']
## 4. 数据预处理设置
# 4.1. 降采样：1920 Hz -- 1000 Hz
# 4.2. emg陷波滤波（3阶FIR陷波滤波器，50Hz，频带宽度 2Hz）
# 4.3. emg巴特沃斯滤波(低截止频率:15hz;高截止频率:450hz; 阶数:7)
# 4.4. emg去噪
## 4.4.1 ICEEMDAN-PE-GT(garrote阈值，即改进阈值)/SVD(信号的主要成分会对应较大的奇异值，而噪声则对应较小的奇异值, 通过设定一个阈值，将较小的奇异值置为零，从而抑制噪声对信号的影响, 阈值可以根据奇异值的突变点来确定，突变点之后的奇异值可以被视为噪声，而突变点之前的奇异值则代表了信号的主要成分)
## 4.4.1.1 ICEEMDAN：Noise amplitude，ε0 = 0.2； Number of the noise realizations， I=100；
## 4.4.1.2 自适应的PE判断阈值：1）将一阶IMF作为噪声序列X，计算其PE，其中含噪IMF的阶数为m = 1；
##                          2）令m = m + 1，重构第m阶IMF与噪声序列X;将重构后的序列视为新的噪声序列X，计算其PE
##                          3) 若噪声序列的PE增大，继续重复步骤2);如果PE减小，则停止判断。我们把当前m阶之前的IMF视为嘈杂的IMF;也就是说，IMF（k=1, .., m）是噪声IMF
## 4.4.1.3 GT: 阈值：通用阈值'universal'(即visushrink)的改进； 阈值规则：garrote阈值
## 4.4.2 WD-GT(小波去噪，小波基“db5”, 分解层数3, 阈值规则：garrote阈值, 阈值通用阈值'universal')
## 4.4.3 WPD-GT(小波包去噪，小波基“db5”, 分解层数3, 阈值规则：garrote阈值, 阈值通用阈值'universal')
# 4.5. emg归一化
raw_fs, tar_fs, imf_band, imf_freq, lowcut, highcut  = 1920, 1000, 2, 50, 15, 450
emg_denoise_method = 'EMD-PE-SVD'  # ['rawdata', 'WD-GT', 'WPD-GT', 'EMD-PE-GT', 'EMD-PE-SVD', 'ICEEMDAN-PE-GT', 'ICEEMDAN-PE-SVD']
data_normalize_method = 'positive_negative_one' # 支持：'min-max' （0-1之间），'max-abs' (-1,1之间)，'positive_negative_one' （-1，1之间）
data_normalize_level = 'matrix' # 'matrix' （矩阵层面，使用整个矩阵的最大值最小值来对每一列进行操作，这样保持了每种模态信号的相对幅值），
                          # 'rows' （列层面，使用每一列（通道）的最大值最小值来对每一列进行操作，即每一列都被单独归一化）
## 5. 重叠窗口采样：重叠窗长， 步进长度
## 5.1. 对于除UPS. WAK和DNS外的数据，去除休息数据，即标签为R的数据
window = 128
step = int(0.75*window) 
## 6. emg和关节角度数据的特征提取设置
emg_channels = ['TFL', 'RF', 'VM', 'SEM', 'UTA', 'LTA', 'LG', 'MG', 'SOL']
emg_feature_type = ['MAV', 'RMS', 'WL', 'ZC', 'SSC', 'WAMP']
fea_normalize_method = 'min-max'
fea_normalize_level = 'rows'

In [ ]:
### 处理和保存数据
for i in range(len(subjects_list_global)):
    subject = 'Sub'+subjects_list_global[i]
    data_file_dir = os.path.join(raw_file_path,subject,'Data')
    label_file_dir = os.path.join(raw_file_path,subject,'Labels')
    sub_emg_sample, sub_emg_features, sub_motion_label_raw = [], [], []
    printlog(info = ''.join(['开始处理： ', subject]), time = True, line_break = False)
    for j in range(len(motions_list_global)):
        motion = motions_list_global[j]
        printlog(info = "\n"+"********"*4 +''.join([' motion: ', motion, ' '])+"********"*4, time = False, line_break = False)
        printlog (info = 'step1: 加载data和label文件...', time = False, line_break = False)
        ## 1.1 data和label文件名
        data_file_name = os.path.join(data_file_dir,''.join([subject, '_', motion, '_Data.csv']))
        label_file_name = os.path.join(label_file_dir,''.join([subject, '_', motion, '_Label.csv']))
        ## 1.2 read data and label
        df_data = pd.read_csv(data_file_name,header=0)
        time = df_data.loc[:, ['Time']].values
        emg_raw_data = df_data.loc[:, emg_name].values
        df_label = pd.read_csv(label_file_name,header=0)
        status_label = df_label.loc[:, ['Status']].values
        group_label = df_label.loc[:, ['Group']].values
        printlog ('step2: 数据预处理...', time = False, line_break = False)
        ## 2.0 判断是否需要活动段信号
        emg_act_data = get_emg_act_signal(motion, emg_raw_data, status_label)
        ## 2.1 降采样
        emg_resample_data = signal2d_resampling(emg_act_data, raw_fs, tar_fs)
        # print(emg_raw_data.shape, emg_act_data.shape, emg_resample_data.shape)
        
        ## 2.2 滤波
        cows, rows = emg_resample_data.shape[0],  emg_resample_data.shape[1] #shape: datapoint*channel
        emg_preprocessing = emg_filtering(tar_fs, lowcut, highcut, imf_band, imf_freq)
        ## 2.2.1 emg陷波滤波
        emg_inf_data = np.zeros((cows, rows))
        for i in range(rows):
            emg_inf_data[:, i] = emg_preprocessing.Implement_Notch_Filter(emg_resample_data[:, i])
        ## 2.2.2 emg butterWorth带通滤波
        emg_bpf_data = np.zeros((cows, rows))
        for i in range(rows):
            emg_bpf_data[:, i] = emg_preprocessing.butter_bandpass_filter(emg_inf_data[:, i])
        ## 2.3 emg 去噪
        emg_denoise_data = Signal2dDenoise(data=emg_bpf_data, denoise_method=emg_denoise_method).forward()
        # print(emg_denoise_data.shape, emg_bpf_data.shape)
        ## 2.4 emg的归一化
        emg_data_pre = data_nomalize(emg_denoise_data, data_normalize_method, data_normalize_level)
        printlog ('step3: 活动段提取、重叠窗口分割...', time = False, line_break = False)
        emg_sample, motion_label_raw = movement_classification_sample_segmentation(motion, emg_data_pre, window, step)
        printlog ('step4: emg特征提取...', time = False, line_break = False)
        emg_features, emg_feature_list = emg_feature_extraction(emg_sample, emg_channels, emg_feature_type, fea_normalize_method, fea_normalize_level)
        printlog ('step5: 合并：所有的emg数据、emg特征、标签...', time = False, line_break = False)
        sub_emg_sample.extend(emg_sample)
        sub_emg_features.extend(emg_features)
        sub_motion_label_raw.extend(motion_label_raw)

    printlog(info = ''.join(['合并受试者：', subject, '所有运动类型的数据...']), time = True, line_break = False)
    sub_emg_sample = np.swapaxes(np.array(sub_emg_sample), 1, 2)   # 使用 np.swapaxes 将第二维和第三维进行交换，变为num*len(emg_channels)*len(window)
    sub_emg_features, sub_motion_label_raw = np.array(sub_emg_features), np.array(sub_motion_label_raw)
    printlog(info = ''.join(['获取受试者：', subject, '所有运动类型的十进制标签和one-hot标签...']), time = False, line_break = False)
    ## 创建并使用 LabelEncoder 对象，并使用 fit_transform 方法进行编码
    label_encoder = LabelEncoder()
    sub_motion_label_encoded_raw = label_encoder.fit_transform(sub_motion_label_raw) ## 默认会根据标签的字母顺序为它们分配编码
    # 为了根据自定义顺序重新排列标签，可以使用 classes_ 属性的索引来获取标签所对应的编码，然后使用 transform 方法进行编码
    custom_order = motions_list_global
    label_encoder.classes_ = custom_order
    sub_motion_label_encoded = np.array([custom_order.index(label) for label in sub_motion_label_raw])
    ## 创建并使用 OneHotEncoder 对象，并使用 fit_transform 方法进行编码
    onehot_encoder = OneHotEncoder(sparse_output=False) #  sparse '在1.2中已弃用，并将在1.4中删除，使用' sparse_output '代替
    sub_motion_label_onehot = onehot_encoder.fit_transform(sub_motion_label_encoded.reshape(-1, 1))
    printlog(info = ''.join(['受试者：', subject, ', ', '处理完毕！']), time = False, line_break = False)
    print('emg_sample.shape: ', sub_emg_sample.shape)
    print('emg_features.shape: ', sub_emg_features.shape)
    print('motion_label_raw.shape: ', sub_motion_label_raw.shape, ', motion_label_encoded.shape: ', sub_motion_label_encoded.shape, ', motion_label_onehot.shape: ', sub_motion_label_onehot.shape)
    printlog(info = ''.join(['保存受试者：', subject, '数据...']), time = True, line_break = False)
    ## 获取路径和文件名
    file_save_path = os.path.join(os.getcwd(), 'trainData', emg_denoise_method)
    make_dir(file_save_path)
    file_save_name = os.path.join(file_save_path,''.join([subject,'_targetTrainData.npz']))
    ## 保存文件
    with open(file_save_name, 'wb') as f:
        np.savez(f, sub_emg_sample = sub_emg_sample, sub_emg_features = sub_emg_features, sub_motion_label_raw = sub_motion_label_raw,
        sub_motion_label_encoded = sub_motion_label_encoded, sub_motion_label_onehot = sub_motion_label_onehot)
    printlog(info = ''.join(['受试者: ',subject, '数据保存完毕！']), time = False, line_break = False)
    printlog(info = file_save_name, time = False, line_break = False)